In [37]:
import chromadb 

host='localhost'
port='8000'
client = chromadb.HttpClient(host=host, port=port) 


client.list_collections()

[Collection(name=langchain), Collection(name=resume)]

In [5]:
data =  {
        "source": {
            "id": None,
            "name": "Threadreaderapp.com"
        },
        "author": None,
        "title": "Walls are starting to close in for Tesla, let's have a closer look",
        "description": "@ZeroSumBond: $TSLA Walls are starting to close in for Tesla, let's have a closer look: 1) The recent turnover in senior executives is incredible, with most having a tenure well over 5 years. From CFO, to...\u2026",
        "url": "https://threadreaderapp.com/thread/1787501413137858617.html",
        "urlToImage": "https://threadreaderapp.com/images/screenshots/thread/1787501413137858617.jpg",
        "publishedAt": "2024-05-22T20:22:25Z",
        "content": "Support us! We are indie developers!\r\nThis site is made by just two indie developers on a laptop doing marketing, support and development! Read more about the story.\r\nBecome a Premium Member ($3/mont\u2026 [+395 chars]"
    }

In [20]:
from api_test import Perigon
import os 
import json 
api = Perigon(os.getenv('PERIGON_API_KEY'), 'https://api.goperigon.com/v1/')
# rom=2024-05-03&q="SpaceX" OR "Tesla"&sourceGroup=top100&language=en&apiKey=[KEY]
params = { 
	"from": "2024-05-03",
	"sourceGroup": "top100",
	"language": "en"
}
res = api.all('SpaceX', **params)


https://api.goperigon.com/v1/all?apiKey=6b47d882-a23a-44e1-b615-88c8b833512e&q=SpaceX&from=2024-05-03&sourceGroup=top100&language=en
Successfully Retrieve 1023 articles


In [25]:
print(res['articles'][0]['url'])
print(res['articles'][0]['content'])
print(res['articles'][0]['summary'])

print(res['articles'][0])
f = open('temp.json', 'w')
json.dump(res['articles'][0],f, indent=4, ensure_ascii=True)
f.close()

https://www.bloomberg.com/news/features/2024-05-21/how-to-buy-spacex-stock-here-s-a-pre-ipo-investing-guide?srnd=homepage-europe
The constellation Monoceros, faintly visible against the Milky Way, has for centuries occupied the heavens as “the unicorn.” The other space unicorn? Elon Musk ’s Space Exploration Technologies Corp. ... [1005 symbols]
Elon Musk's Space Exploration Technologies Corp. (SpaceX), the most highly valued startup in the US, has grown into a $180 billion colossus that launches satellites, operates the Starlink internet service, and is a critical NASA subcontractor. However, SpaceX remains privately held and its shares do not currently trade on a stock exchange. As a result, obtaining a piece of SpaceX is a process that can increase its valuation. Investing in SpaceX is not simple, but it can create demand and raise the company's valuation.
{'url': 'https://www.bloomberg.com/news/features/2024-05-21/how-to-buy-spacex-stock-here-s-a-pre-ipo-investing-guide?srnd=homepa

In [26]:
from langchain_community.vectorstores import Chroma
import chromadb.utils.embedding_functions as embedding_functions
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import CharacterTextSplitter

In [39]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

documents = text_splitter.create_documents([res['articles'][0]['summary']],metadatas=[{'url': res['articles'][0]['url']}])

docs = text_splitter.split_documents(documents)

In [44]:
docs[0].str

Document(page_content="Elon Musk's Space Exploration Technologies Corp. (SpaceX), the most highly valued startup in the US, has grown into a $180 billion colossus that launches satellites, operates the Starlink internet service, and is a critical NASA subcontractor. However, SpaceX remains privately held and its shares do not currently trade on a stock exchange. As a result, obtaining a piece of SpaceX is a process that can increase its valuation. Investing in SpaceX is not simple, but it can create demand and raise the company's valuation.", metadata={'url': 'https://www.bloomberg.com/news/features/2024-05-21/how-to-buy-spacex-stock-here-s-a-pre-ipo-investing-guide?srnd=homepage-europe'})

In [43]:
embedding_function = OllamaEmbeddings(model="nomic-embed-text")

db = Chroma.from_documents(docs[0], embedding_function, client=client, collection_name='langchain')

AttributeError: 'tuple' object has no attribute 'page_content'

In [ ]:
Chroma.similarity_search(q) 